In [ ]:
#MI with CDI Paper method (Faiz, M. A. et al. A composite drought index developed for detecting large-scale drought characteristics. Journal of Hydrology 605, 127308 (2022))
import xarray as xr
import numpy as np

# Function to cap MI values at -1 and +1
def cap_mi_values(array):
    array = np.where(array < -1, -1, array)
    array = np.where(array > 1, 1, array)
    return array

# Load the precipitation data from NetCDF file
precipitation_nc = xr.open_dataset('PR_Monthly_2000_23.nc')

# Load the evapotranspiration data from NetCDF file
evapotranspiration_nc = xr.open_dataset('2000_2023_monthly_E.nc')

# Ensure both datasets have the same time range and spatial resolution
precipitation_nc = precipitation_nc.sel(time=slice('2000-01-01', '2022-12-31'))
evapotranspiration_nc = evapotranspiration_nc.sel(time=slice('2000-01-01', '2022-12-31'))

# Interpolate evapotranspiration data to match the precipitation data grid
evapotranspiration_nc = evapotranspiration_nc.interp(lat=precipitation_nc.lat, lon=precipitation_nc.lon, method="nearest")

# Calculate the mean precipitation and mean evapotranspiration over the entire period
mean_precipitation = precipitation_nc['precipitation'].mean(dim='time')
mean_evapotranspiration = evapotranspiration_nc['E'].mean(dim='time')

# Calculate the standard MI for each month for each grid point using the new formula
mi_standard = (precipitation_nc['precipitation'] - evapotranspiration_nc['E']) / (mean_precipitation - mean_evapotranspiration)

# Apply the capping function to the MI values
mi_standard_corrected = cap_mi_values(mi_standard)

# Create a new xarray Dataset for the corrected MI
mi_standard_corrected_ds = xr.Dataset(
    {
        "MI_corrected": (["time", "lat", "lon"], mi_standard_corrected)
    },
    coords={
        "time": precipitation_nc.time,
        "lat": precipitation_nc.lat,
        "lon": precipitation_nc.lon
    }
)

# Save the corrected MI dataset to a new NetCDF file
corrected_mi_filename = '/MI_CDI_Method.nc'
mi_standard_corrected_ds.to_netcdf(corrected_mi_filename)

print("Corrected Moisture Index has been calculated and saved to the NetCDF file.")


In [ ]:
#MI with Chinese paper Method (Zhou, W., Liu, G., Pan, J., Feng, X., 2005. Distribution of available soil water capacity in China. J. Geog. Sci. 15 (1), 3–12
#Qian, W., Shan, X., Zhu, Y., 2011. Ranking regional drought events in China for 1960–2009. Advances in Atmospheric Sciences 28, 310–321.
import xarray as xr
import numpy as np

# Function to cap MI values at -1 and +1
def cap_mi_values(array):
    array = np.where(array < -1, -1, array)
    array = np.where(array > 1, 1, array)
    return array

# Load the precipitation data from NetCDF file
precipitation_nc = xr.open_dataset('/PR_Monthly_2000_23.nc')

# Load the evapotranspiration data from NetCDF file
evapotranspiration_nc = xr.open_dataset('/2000_2023_monthly_E.nc')

# Ensure both datasets have the same time range and spatial resolution
precipitation_nc = precipitation_nc.sel(time=slice('2000-01-01', '2022-12-31'))
evapotranspiration_nc = evapotranspiration_nc.sel(time=slice('2000-01-01', '2022-12-31'))

# Interpolate evapotranspiration data to match the precipitation data grid
evapotranspiration_nc_interp = evapotranspiration_nc.interp(lat=precipitation_nc.lat, lon=precipitation_nc.lon, method="nearest")

# Avoid division by zero in MI calculation
precipitation_values = precipitation_nc['precipitation'].values
evapotranspiration_values = evapotranspiration_nc_interp['E'].values

# Replace zero precipitation values with a small number (e.g., 1e-10) to avoid division by zero
precipitation_values = np.where(precipitation_values == 0, 1e-10, precipitation_values)

# Calculate the standard MI for each month for each grid point
mi_standard = (precipitation_values - evapotranspiration_values) / precipitation_values

# Apply the capping function to the MI values
mi_standard_corrected = cap_mi_values(mi_standard)

# Create a new xarray Dataset for the corrected MI
mi_standard_corrected_ds = xr.Dataset(
    {
        "MI_corrected": (["time", "lat", "lon"], mi_standard_corrected)
    },
    coords={
        "time": precipitation_nc.time,
        "lat": precipitation_nc.lat,
        "lon": precipitation_nc.lon
    }
)

# Save the corrected MI dataset to a new NetCDF file
corrected_mi_filename = '/MI__Method.nc'
mi_standard_corrected_ds.to_netcdf(corrected_mi_filename)

print("Corrected Moisture Index has been calculated and saved to the NetCDF file.")


In [1]:
#CMDI Final with PCA (This is CDI_Main calculation) for our own settings we named them like this......
#Please for accurate abbreviation, consult our published papers (A composite drought index developed for detecting large-scale drought characteristics;
#Revisiting the composite drought index for improving drought monitoring
import numpy as np
import netCDF4 as nc
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# Function to trim and reshape data
def trim_and_reshape(data, common_shape):
    trimmed_data = data[:common_shape[0], :common_shape[1], :common_shape[2]]
    return trimmed_data.flatten()

# Function to process and combine indices using PCA
def process_chunk(mRAI_chunk, mMRA2_chunk, MI_chunk, common_shape):
    mRAI_1d = trim_and_reshape(mRAI_chunk, common_shape)
    mMRA2_1d = trim_and_reshape(mMRA2_chunk, common_shape)
    MI_1d = trim_and_reshape(MI_chunk, common_shape)

    # Combine data
    data_combined = np.column_stack((mRAI_1d, mMRA2_1d, MI_1d))
    
    # Impute missing values
    imputer = SimpleImputer(strategy='mean')
    data_imputed = imputer.fit_transform(data_combined)
    
    # Apply PCA
    pca = PCA(n_components=1)
    cmdi_pca = pca.fit_transform(data_imputed).flatten()

    return cmdi_pca.reshape(common_shape)

# Load data from NetCDF files
mRAI_path = "/.nc" # is mRAI
mMRA2_path = "/.nc" # here it is mWBAI
MI_path = "/.nc" #MI 

mRAI_data = nc.Dataset(mRAI_path)
mMRA2_data = nc.Dataset(mMRA2_path)
MI_data = nc.Dataset(MI_path)

# Extract variables
mRAI = mRAI_data.variables['mRAI']
mMRA2 = mMRA2_data.variables['mRAI']
MI = MI_data.variables['MI_corrected']

# Determine the common shape
common_shape = (
    min(mRAI.shape[0], mMRA2.shape[0], MI.shape[0]),
    min(mRAI.shape[1], mMRA2.shape[1], MI.shape[1]),
    min(mRAI.shape[2], mMRA2.shape[2], MI.shape[2])
)

print(f"Common shape: {common_shape}")

# Create a new NetCDF file to save CMDI
output_path = "/CMDI_Main.nc"
with nc.Dataset(output_path, 'w', format='NETCDF4') as cmdi_nc:
    # Create dimensions
    cmdi_nc.createDimension('time', common_shape[0])
    cmdi_nc.createDimension('lat', common_shape[1])
    cmdi_nc.createDimension('lon', common_shape[2])
    
    # Create variables
    times = cmdi_nc.createVariable('time', 'f8', ('time',))
    lats = cmdi_nc.createVariable('lat', 'f8', ('lat',))
    lons = cmdi_nc.createVariable('lon', 'f8', ('lon',))
    cmdi_var = cmdi_nc.createVariable('CMDI', 'f8', ('time', 'lat', 'lon'))
    
    # Assign dimension data
    times[:] = MI_data.variables['time'][:common_shape[0]]
    lats[:] = MI_data.variables['lat'][:common_shape[1]]
    lons[:] = MI_data.variables['lon'][:common_shape[2]]

    # Process data in chunks to save memory
    chunk_size = 10  # Adjust this to control memory usage
    for start in range(0, common_shape[0], chunk_size):
        end = min(start + chunk_size, common_shape[0])
        mRAI_chunk = mRAI[start:end, :common_shape[1], :common_shape[2]]
        mMRA2_chunk = mMRA2[start:end, :common_shape[1], :common_shape[2]]
        MI_chunk = MI[start:end, :common_shape[1], :common_shape[2]]
        
        cmdi_chunk = process_chunk(mRAI_chunk, mMRA2_chunk, MI_chunk, (end-start, common_shape[1], common_shape[2]))
        cmdi_var[start:end, :, :] = cmdi_chunk

    # Add attributes
    cmdi_nc.description = "Composite Drought Index (CMDI)" #CMDI is refer to CDI, we used differnt variables due to lengthy data calculations for multiple files 
    times.units = MI_data.variables['time'].units
    lats.units = 'degrees_north'
    lons.units = 'degrees_east'
    cmdi_var.units = 'unitless'

print("CMDI data saved to NetCDF file successfully.")


Common shape: (264, 720, 1440)
CMDI data saved to NetCDF file successfully.


In [12]:
#CDI Weights multiply due to computation limitations 
import xarray as xr

# File paths
mRAI_path = "/.nc" # this mRAI
mMRA2_path = "/.nc" #This is mBAI
MI_path = "/.nc" # This MI

# Open the NetCDF files
mRAI_ds = xr.open_dataset(mRAI_path)
mMRA2_ds = xr.open_dataset(mMRA2_path)
MI_ds = xr.open_dataset(MI_path)

# Extract the relevant data arrays
mRAI_data = mRAI_ds['mRAI']
mMRA2_data = mMRA2_ds['mRAI']
MI_data = MI_ds['MI_corrected']

# Define the common time period
start_date = "2000-01-01"
end_date = "2021-12-31"

# Subset the data to the common time period
mRAI_data = mRAI_data.sel(time=slice(start_date, end_date))
mMRA2_data = mMRA2_data.sel(time=slice(start_date, end_date))
MI_data = MI_data.sel(time=slice(start_date, end_date))

# Apply the given weights
mRAI_weighted = mRAI_data * 0.4
mMRA2_weighted = mMRA2_data * 0.4
MI_weighted = MI_data * 0.8

# Save the weighted data arrays to new NetCDF files
mRAI_weighted_ds = xr.Dataset({"mRAI_weighted": mRAI_weighted})
mRAI_weighted_ds.to_netcdf("/mRAI_weighted.nc")

mMRA2_weighted_ds = xr.Dataset({"mMRA2_weighted": mMRA2_weighted})
mMRA2_weighted_ds.to_netcdf("/mMRA2_weighted.nc")

MI_weighted_ds = xr.Dataset({"MI_weighted": MI_weighted})
MI_weighted_ds.to_netcdf("/MI_weighted.nc")

print("Weighted files saved successfully.")


Weighted files saved successfully.


In [16]:
#CDI Weight Combine to make CMDI (CDI)
import netCDF4 as nc
import numpy as np

# File paths
mRAI_weighted_file = '/mRAI_weighted.nc'
mMRA2_weighted_file = '/mMRA2_weighted.nc'
MI_weighted_file = '/MI_weighted.nc'
CMDI_file = '/CDI.nc'

# Open the NetCDF files
mRAI_ds = nc.Dataset(mRAI_weighted_file, 'r')
mMRA2_ds = nc.Dataset(mMRA2_weighted_file, 'r')
MI_ds = nc.Dataset(MI_weighted_file, 'r')

# Create a new NetCDF file
CMDI_ds = nc.Dataset(CMDI_file, 'w', format='NETCDF4')

# Define dimensions
time_dim = CMDI_ds.createDimension('time', 264)
lat_dim = CMDI_ds.createDimension('lat', 721)
lon_dim = CMDI_ds.createDimension('lon', 1441)

# Create variables
time_var = CMDI_ds.createVariable('time', np.float64, ('time',))
lat_var = CMDI_ds.createVariable('lat', np.float64, ('lat',))
lon_var = CMDI_ds.createVariable('lon', np.float64, ('lon',))
CMDI_var = CMDI_ds.createVariable('CMDI', np.float64, ('time', 'lat', 'lon',), fill_value=np.nan)

# Assign attributes
time_var.units = "days since 2000-01-31"
time_var.calendar = "proleptic_gregorian"
lat_var.units = "degrees_north"
lon_var.units = "degrees_east"

# Read data from the source files
time_data = mRAI_ds.variables['time'][:]
lat_data = mRAI_ds.variables['lat'][:]
lon_data = mRAI_ds.variables['lon'][:]
mRAI_data = mRAI_ds.variables['mRAI_weighted'][:]

# Handle different dimensions for mMRA2
mMRA2_data_resized = np.full((264, 721, 1441), np.nan)
mMRA2_data = mMRA2_ds.variables['mMRA2_weighted'][:]

# Resize mMRA2 data to match the dimensions of other datasets
for t in range(263):
    mMRA2_data_resized[t, :720, :1440] = mMRA2_data[t, :, :]

MI_data = MI_ds.variables['MI_weighted'][:]

# Combine the data
CMDI_data = mRAI_data + mMRA2_data_resized + MI_data

# Assign data to variables
time_var[:] = time_data
lat_var[:] = lat_data
lon_var[:] = lon_data
CMDI_var[:] = CMDI_data

# Close the datasets
mRAI_ds.close()
mMRA2_ds.close()
MI_ds.close()
CMDI_ds.close()

print("New NetCDF file CMDI.nc created successfully.")


New NetCDF file CMDI.nc created successfully.
